***Import necessary libraries***

In [1]:
import rioxarray as rxr
import os
from rasterio.plot import show
from PIL import Image
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import geopandas as gpd
import rasterio.features
from shapely.geometry import shape
import pandas as pd
import re

Lets create the files variables

In [2]:
# retreive some infos on fully contained segments
contained_file_50 = 'data/contained_segments_50.txt'
contained_file_0001 = 'data/contained_segments_0001.txt'
contained_file_100 = 'data/contained_segments_100.txt'

# Create necessary functions

First lets create the function that transform the file of contained segments into a dataframe.

In [89]:
def open_included_segments(contained_file):
    # load the data
    with open(contained_file) as f:
        lines = f.readlines()
        # reteive line infos
        segment_id = []
        polygon_id = []
        class_id = []
        pixels = []
        for line in lines:
            # retrieve segment id
            segment_id.append(line.split(',')[0].split(':')[1].strip())
            # retrieve polygon id
            polygon_id.append(line.split(',')[1].split(':')[1].strip())
            # retrieve class id
            class_id.append(line.split(',')[2].split(':')[1].strip())
            # retrieve pixels
            pixel_list_str = line.split(':')[4]
            pixel_list_str = pixel_list_str[3:-3]
            pixel_list = list(pixel_list_str.split('], ['))
            pixel_list = [pixel.split(', ') for pixel in pixel_list]
            pixel_list = [[int(pixel[0]), int(pixel[1])] for pixel in pixel_list]
            pixels.append(pixel_list)
        
        # create a dataframe
        df_full = pd.DataFrame()
        df_full['segment_id'] = segment_id
        df_full['polygon_id'] = polygon_id
        df_full['class_id'] = class_id
        df_full['pixels'] = pixels
        return df_full

Now create the function that shows infos on the included segments and display the distribution of the classes and the distributino of segments per polygon.  

In [ ]:
def show_info_on_segments(df):
    # show repartition, infos
    print('Number of segments: ', len(df))
    # show mean number of pixels per segment
    print('Mean number of pixels per segment: ', np.mean([len(pixels) for pixels in df['pixels']]))

    print('Number of fully contained segments per class: ')
    print(df.groupby('class_id').count()['segment_id'])
    # show the repartition using histogram
    plt.hist(df['class_id'])
    plt.show()

    print('Number of fully contained segments per polygon: ')
    #print(fully_contained_segments_df.groupby('polygon_id').count()['segment_id'])
    # show the repartition using histogram
    plt.hist(df['polygon_id'])
    # remove x labels
    plt.xticks([])
    plt.show()

# Use the function

In [ ]:
dataframe_50 = open_included_segments(contained_file_50)
dataframe_0001 = open_included_segments(contained_file_0001)
dataframe_100 = open_included_segments(contained_file_100)

show_info_on_segments(dataframe_50)
show_info_on_segments(dataframe_0001)
show_info_on_segments(dataframe_100)

Results are the following :
* **Results for `0.001%`**  
Number of segments:  8916
Mean number of pixels per segment:  16.270300583221175
Number of fully contained segments per class:
class_id
1.0    1364
2.0     925
3.0    1277
4.0    1436
5.0    1986
6.0    1597
7.0     203
8.0     128  
<img src="assets\segment_0001_1.png" width=500>
<img src="assets\segment_0001_2.png" width=500>

* **Results for `50%`**  
Number of fully contained segments:  4794
Number of fully contained segments per class:
class_id
1.0     544
2.0     409
3.0     427
4.0     794
5.0    1403
6.0    1089
7.0      56
8.0      72  
<img src="assets\segment_50_1.png" width=500>
<img src="assets\segment_50_2.png" width=500>

* **Results for `100%`**  
Number of fully contained segments:  2324
Number of fully contained segments per class:
class_id
1.0    165
2.0    125
3.0     74
4.0    334
5.0    920
6.0    661
7.0      8
8.0     37  
<img src="assets\segment_100_1.png" width=500>
<img src="assets\segment_100_2.png" width=500>


That lead to the conclusion that we should use 50% for the rest of the project, as it is a good compromise between the number of segments and the class distribution.

Next step is to normalize the raster images.